In [1]:
import random
import csv
import os
import pickle
import sys
os.chdir('Resources/')

In [2]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [3]:
def split_message(message, max_size):
    encoded = message.encode()
    chunks = [encoded[i:i + max_size] for i in range(0, len(encoded), max_size)]
    return [int.from_bytes(chunk, 'big') for chunk in chunks]

In [4]:
def join_chunks(chunks, max_size):
    """Reassemble integer chunks into the original message."""
    message_bytes = b''.join(chunk.to_bytes(max_size, 'big') for chunk in chunks)
    return message_bytes.decode(errors='ignore')

In [5]:
def verify_signature(signed_chunks, original_message, public_key):
    """Verify the RSA signature by decrypting and comparing."""
    e, n = public_key
    max_bytes = (n.bit_length() // 8) - 1

    original_chunks = split_message(original_message, max_bytes)

    decrypted_chunks = [power(sig, e, n) for sig in signed_chunks]

    return original_chunks == decrypted_chunks

In [6]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [7]:
sys_rsa_2_public_key = load_key("17_SYS_RSA_2_Public_Key.pkl")
sys_rsa_2_public_key

(2122126697722114073, 4088522690914825687)

In [8]:
with open("19_(SYS)_SU_Transmitted_User_Data.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    rows = list(reader)

encrypted_aes_key = rows[0][0]
encrypted_data = rows[1][0]
signed_chunks = eval(rows[2][0])

print(encrypted_aes_key, '\n', encrypted_data, '\n', signed_chunks)

2136287171331916797 
 áâøåøæøåàäøæáäøåøæøåáäøäøæøå 
 [882643961481838630, 1337866713467690038, 3140871930271757227, 4059081381053974844, 2210007273399152650, 2674233172039042455, 2940767274681283384, 2100575981009037332, 2210007273399152650, 2674233172039042455, 46090542140770433, 1624928314837446202, 3199564187534310285]


In [9]:
original_message = str(encrypted_aes_key) + '\n' + encrypted_data
is_valid = verify_signature(signed_chunks, original_message, sys_rsa_2_public_key)

print("Signature Verification:", "Valid" if is_valid else "Invalid")

Signature Verification: Valid


In [10]:
sys_rsa_1_private_key = load_key("16_SYS_RSA_1_Private_Key.pkl")
sys_rsa_1_private_key

(3072872903477649361, 4936349149105973713)

In [11]:
def decrypt_data_with_aes(encrypted_data, key):
    decrypted_data = "".join(chr(ord(c) ^ key) for c in encrypted_data)
    return decrypted_data

In [12]:
def decrypt_aes_key_with_rsa(encrypted_aes_key, private_key):
    d, n = private_key
    return power(encrypted_aes_key, d, n)

In [13]:
def decrypt_and_map_csv(input_filename, private_key, output_filename):
    with open(input_filename, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = list(reader)

    encrypted_aes_key = int(rows[0][0])  
    encrypted_data = rows[1][0]  

    aes_key = decrypt_aes_key_with_rsa(encrypted_aes_key, private_key)
    decrypted_data = decrypt_data_with_aes(encrypted_data, aes_key)

    column_names = [
        "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol",
        "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina",
        "Oldpeak", "ST_Slope", "HeartDisease"
    ]

    mappings = {
        1: {0: 'Female', 1: 'Male'},
        2: {0: 'Typical Angina', 1: 'Atypical Angina', 2: 'Non-anginal Pain', 3: 'Asymptomatic'},
        5: {0: 'No', 1: 'Yes'},
        6: {0: 'Normal', 1: 'Abnormal ST-T Wave', 2: 'Left Ventricular Hypertrophy'},
        8: {0: 'No', 1: 'Yes'},
        10: {0: 'Upsloping', 1: 'Flat', 2: 'Downsloping'}
    }

    decrypted_rows = [column_names] + [row.split(",") for row in decrypted_data.split("\n")]

    for row in decrypted_rows[1:]:  # Skip header
        for col, mapping in mappings.items():
            row[col] = mapping.get(int(row[col]), row[col])

    with open(output_filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(decrypted_rows)

    print(f"Decrypted and mapped data saved as '{output_filename}'.")

In [14]:
if is_valid:
    output_filename = '20_(SYS)_CC_Received_User_Data.csv'
    decrypt_and_map_csv("19_(SYS)_SU_Transmitted_User_Data.csv", sys_rsa_1_private_key, output_filename)
else:
    print("Invalid User Data")

Decrypted and mapped data saved as '20_(SYS)_CC_Received_User_Data.csv'.
